In [ ]:
from pyspark.sql.functions import col

In [ ]:
# Check if the mount point already exists
mount_point = "/mnt/2021olympics"
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)

# Define the configuration
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "523fc33b-fffc-4f4e-a09e-460d8e60e0af",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/b9b9fed5-914c-4f13-9dd7-80958b6b6b9a/oauth2/token"}

# Mount the storage
dbutils.fs.mount(
source = "abfss://2021-olympic-data@2021olympicsstorage.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/2021olympics",
extra_configs = configs)

/mnt/2021olympics has been unmounted.


True

In [ ]:
%fs ls "/mnt/2021olympics/raw-data"

path,name,size,modificationTime
dbfs:/mnt/2021olympics/raw-data/athletes.csv,athletes.csv,418492,1723181002000
dbfs:/mnt/2021olympics/raw-data/coaches.csv,coaches.csv,16889,1723181016000
dbfs:/mnt/2021olympics/raw-data/entriesgender.csv,entriesgender.csv,1123,1723249232000
dbfs:/mnt/2021olympics/raw-data/medals.csv,medals.csv,2414,1723249230000
dbfs:/mnt/2021olympics/raw-data/teams.csv,teams.csv,35270,1723181054000


In [ ]:
athletes = spark.read.format("csv").option("header", "true").load("/mnt/2021olympics/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header", "true").load("/mnt/2021olympics/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header", "true").load("/mnt/2021olympics/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header", "true").load("/mnt/2021olympics/raw-data/medals.csv")
teams = spark.read.format("csv").option("header", "true").load("/mnt/2021olympics/raw-data/teams.csv")

In [ ]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [ ]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [ ]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
#Convert string values to integers
entriesgender = entriesgender.withColumn("Female", col("Female").cast("int"))\
        .withColumn("Male", col("Male").cast("int"))\
        .withColumn("Total", col("Total").cast("int"))

In [ ]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
#Convert string values to integers
medals = medals.withColumn("Gold", col("Gold").cast("int")) \
               .withColumn("Silver", col("Silver").cast("int")) \
               .withColumn("Bronze", col("Bronze").cast("int")) \
               .withColumn("Total", col("Total").cast("int")) \
               .withColumn("Rank by Total", col("Rank by Total").cast("int"))

In [ ]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [ ]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
#Show the countries with the most gold medals
top_gold_medalists = medals.select("TeamCountry", "Gold").orderBy("Gold", ascending=False)
top_gold_medalists.show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|              Canada|   7|
|              Brazil|   7|
|         New Zealand|   7|
|                Cuba|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|              Poland|   4|
|      Czech Republic|   4|
|               Kenya|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [ ]:
#Show the countries with the most medals
top_medalists = medals.select("TeamCountry", "Total").orderBy("Total", ascending=False)
top_medalists.show()

+--------------------+-----+
|         TeamCountry|Total|
+--------------------+-----+
|United States of ...|  113|
|People's Republic...|   88|
|                 ROC|   71|
|       Great Britain|   65|
|               Japan|   58|
|           Australia|   46|
|               Italy|   40|
|             Germany|   37|
|         Netherlands|   36|
|              France|   33|
|              Canada|   24|
|              Brazil|   21|
|         New Zealand|   20|
|             Hungary|   20|
|   Republic of Korea|   20|
|             Ukraine|   19|
|               Spain|   17|
|                Cuba|   15|
|              Poland|   14|
|         Switzerland|   13|
+--------------------+-----+
only showing top 20 rows



In [ ]:
#Find the proportion of female and male participants by discipline
avg_gender_entries = entriesgender.withColumn("Avg Female", col("Female")/col("Total"))\
    .withColumn("Avg Male", col("Male")/col("Total"))

avg_gender_entries.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg Female|           Avg Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [ ]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
#Top countries with the most participants
most_athletes = athletes.groupBy("Country").count().orderBy("count", ascending=False)
most_athletes.show()

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|United States of ...|  615|
|               Japan|  586|
|           Australia|  470|
|People's Republic...|  401|
|             Germany|  400|
|              France|  377|
|              Canada|  368|
|       Great Britain|  366|
|               Italy|  356|
|               Spain|  324|
|                 ROC|  318|
|              Brazil|  291|
|         Netherlands|  274|
|   Republic of Korea|  223|
|         New Zealand|  202|
|              Poland|  195|
|           Argentina|  180|
|        South Africa|  171|
|              Mexico|  155|
|             Hungary|  155|
+--------------------+-----+
only showing top 20 rows



In [ ]:
#Top countries with the least participants
least_athletes = athletes.groupBy("Country").count().orderBy("count", ascending=True)
least_athletes.show()

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|            Dominica|    2|
|             Andorra|    2|
|             Lesotho|    2|
|             Myanmar|    2|
|   Brunei Darussalam|    2|
|             Somalia|    2|
|    Marshall Islands|    2|
|              Tuvalu|    2|
|United Republic o...|    2|
|St Vincent and th...|    2|
|          Mauritania|    2|
|Central African R...|    2|
|               Nauru|    2|
|Saint Kitts and N...|    2|
|         South Sudan|    2|
|             Bermuda|    2|
|             Vanuatu|    2|
|             Comoros|    3|
|               Yemen|    3|
|            Kiribati|    3|
+--------------------+-----+
only showing top 20 rows



In [ ]:
#Save transformed datasets to "transformed-data" folder in Datalake Gen2
athletes.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/2021olympics/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/2021olympics/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/2021olympics/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/2021olympics/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/2021olympics/transformed-data/teams")
